## Data preparation and sanitization

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sb

df = pd.read_excel("../data/HH_SURVEY1/UG_HH_NEW_continuous_no200_train.xls")

df_full = pd.read_excel("../data/UG_HH_NEW_continuous_no200.xls")
df_min = df_full.min()
df_max = df_full.max()

#Drop 'protected' features
drop_cols = ['prop', 'other_prop', 'other_resp']
df = df[[col for col in df if col not in drop_cols]]

df.head()

,time_spent_risk,cells,selfish,time_spent_prop,count_effort,Honesty_Humility,Extraversion,Agreeableness,min_offer
0,43000,26,60,31000,20,4.2,3.0,2.8,50
1,12000,7,20,38000,20,1.9,4.4,2.9,50
2,88000,50,20,69000,12,3.3,3.8,3.4,80
3,24000,50,60,26000,14,3.6,3.8,2.6,50
4,137000,24,60,43000,4,3.6,2.5,2.7,100


## Individual plots

In [2]:
# sb.pairplot(df, x_vars=('prop','other_resp','other_prop'), y_vars='min_offer', height=7, aspect=0.7, kind='reg')
# sb.pairplot(df, x_vars=('cells', 'selfish','count_effort'), y_vars='min_offer', height=7, aspect=0.7)
# sb.pairplot(df, x_vars=('Honesty_Humility', 'Extraversion','Agreeableness'), y_vars='min_offer', height=7, aspect=0.7)

## Determine significant features

In [3]:
from utils.preprocessing import df_to_xy, df_to_xydf
from models import AcceptanceModel
from utils.benchmark import process_model, process_benchmark_cv

res = {}
features = list(df) + ['risk']
if 'min_offer' in features:
    features.remove('min_offer')
for col in features:
    if col == 'risk':    
        x, y = df_to_xy(df, select_columns=[col], fuse_risk=True, df_min=df_min, df_max=df_max)
        col = 'risk*'
    else:
        x, y = df_to_xy(df, select_columns=[col], df_min=df_min, df_max=df_max)
    model = AcceptanceModel()
    item_res = process_benchmark_cv(model, X=x, y=y)
    res[col] = item_res.mean()

res_single_df = pd.DataFrame(res).T
res_single_df.sort_values(by=['avg_loss_ratio'], inplace=True)
res_single_df

Using TensorFlow backend.


,avg_loss,avg_loss_ratio,avg_win_loss,loss_sum,mse,rejection_ratio
selfish,26.979167,0.217393,24.542857,428.0,1482.979167,0.076563
time_spent_prop,27.791667,0.223846,25.461905,441.0,1547.666667,0.060937
Honesty_Humility,28.291667,0.229534,25.019048,449.0,1644.541667,0.069531
cells,29.500000,0.239683,26.338462,468.0,1712.958333,0.074219
Extraversion,30.937500,0.253965,26.541987,490.0,1812.270833,0.083594
time_spent_risk,30.954167,0.256898,26.643727,491.0,1758.895833,0.111583
Agreeableness,31.070833,0.257937,25.888095,493.0,1867.937500,0.085156
risk*,32.329167,0.269230,27.463736,513.0,1930.270833,0.081250
count_effort,33.050000,0.278677,26.314286,525.0,2088.916667,0.121875


In [4]:
from itertools import combinations


features = list(df)
if 'min_offer' in features:
    features.remove('min_offer')

res = {}
for cols in combinations(features, 2):
    #index should be a list
    cols = list(cols)
    if cols[0]==cols[1]: continue
    x, y = df_to_xy(df, select_columns=cols, df_min=df_min, df_max=df_max)
    model = AcceptanceModel()
    item_res = process_benchmark_cv(model, X=x, y=y)
    res[":".join(cols)] = item_res.mean()

res_duo_df = pd.DataFrame(res).T
res_duo_df.sort_values(by=['avg_loss_ratio'], inplace=True)
res_duo_df.head(10)

,avg_loss,avg_loss_ratio,avg_win_loss,loss_sum,mse,rejection_ratio
selfish:time_spent_prop,27.845833,0.224245,25.528571,442.0,1555.937500,0.069260
selfish:Honesty_Humility,28.091667,0.225802,25.780952,446.0,1585.125000,0.092427
cells:selfish,28.620833,0.230273,26.328571,454.0,1613.395833,0.075000
time_spent_risk:cells,29.816667,0.243535,26.492811,473.0,1698.291667,0.114437
selfish:Extraversion,29.995833,0.244543,26.513141,475.0,1695.020833,0.077344
cells:Honesty_Humility,30.554167,0.247828,27.528571,485.0,1798.979167,0.058594
cells:time_spent_prop,31.104167,0.254627,27.337729,494.0,1882.145833,0.075646
time_spent_risk:Agreeableness,31.250000,0.257686,27.936264,495.0,1709.208333,0.084375
cells:Agreeableness,31.537500,0.259373,27.317582,500.0,1862.895833,0.081250
time_spent_prop:Extraversion,31.658333,0.259785,27.512179,502.0,1866.833333,0.084375


In [5]:
from itertools import combinations


features = list(df)
if 'min_offer' in features:
    features.remove('min_offer')

res = {}
for cols in combinations(features, 3):
    #index should be a list
    cols = list(cols)
    if cols[0]==cols[1]: continue
    x, y = df_to_xy(df, select_columns=cols, df_min=df_min, df_max=df_max)
    model = AcceptanceModel()
    item_res = process_benchmark_cv(model, X=x, y=y)
    res[":".join(cols)] = item_res.mean()

res_trio_df = pd.DataFrame(res).T
res_trio_df.sort_values(by=['avg_loss_ratio'], inplace=True)
res_trio_df.head(10)

,avg_loss,avg_loss_ratio,avg_win_loss,loss_sum,mse,rejection_ratio
cells:selfish:time_spent_prop,28.595833,0.229969,26.347619,454.0,1622.520833,0.082542
selfish:time_spent_prop:Extraversion,30.608333,0.249185,27.198443,485.0,1756.000000,0.078125
selfish:time_spent_prop:count_effort,30.470833,0.249826,26.587546,484.0,1842.437500,0.109885
cells:selfish:Honesty_Humility,30.858333,0.250545,27.909524,490.0,1819.333333,0.072521
selfish:Honesty_Humility:Extraversion,30.737500,0.250552,27.286538,487.0,1754.187500,0.095312
time_spent_risk:selfish:time_spent_prop,30.829167,0.254509,27.604762,489.0,1687.270833,0.132917
cells:time_spent_prop:Honesty_Humility,31.312500,0.254675,28.180952,497.0,1837.937500,0.064062
cells:selfish:count_effort,31.241667,0.257060,27.300000,496.0,1852.083333,0.114437
time_spent_risk:selfish:Honesty_Humility,31.387500,0.260229,27.128571,498.0,1798.895833,0.125240
cells:selfish:Extraversion,31.783333,0.261087,27.610897,504.0,1884.000000,0.087500


In [6]:
from itertools import combinations


features = list(df)
if 'min_offer' in features:
    features.remove('min_offer')

res = {}
for cols in combinations(features, 4):
    #index should be a list
    cols = list(cols)
    if cols[0]==cols[1]: continue
    x, y = df_to_xy(df, select_columns=cols, df_min=df_min, df_max=df_max)
    model = AcceptanceModel()
    item_res = process_benchmark_cv(model, X=x, y=y)
    res[":".join(cols)] = item_res.mean()

res_quad_df = pd.DataFrame(res).T
res_quad_df.sort_values(by=['avg_loss_ratio'], inplace=True)
res_quad_df.head(10)

,avg_loss,avg_loss_ratio,avg_win_loss,loss_sum,mse,rejection_ratio
time_spent_risk:selfish:time_spent_prop:Honesty_Humility,30.508333,0.249075,28.276190,484.0,1639.041667,0.107917
cells:selfish:Extraversion:Agreeableness,30.837500,0.250893,27.516804,489.0,1779.270833,0.071094
cells:selfish:time_spent_prop:Extraversion,31.720833,0.257777,28.418040,503.0,1855.020833,0.067969
cells:selfish:time_spent_prop:count_effort,31.537500,0.258621,27.739927,501.0,1910.020833,0.115083
time_spent_risk:time_spent_prop:count_effort:Extraversion,32.070833,0.264046,28.636264,508.0,1787.979167,0.109375
cells:selfish:time_spent_prop:Honesty_Humility,32.433333,0.265816,28.571429,515.0,1971.041667,0.083594
time_spent_risk:selfish:time_spent_prop:Extraversion,32.504167,0.268166,29.286722,515.0,1792.312500,0.083594
selfish:Honesty_Humility:Extraversion:Agreeableness,32.845833,0.269670,28.683333,521.0,1944.354167,0.079687
selfish:time_spent_prop:Extraversion:Agreeableness,32.737500,0.269762,28.597024,519.0,1912.729167,0.080469
cells:time_spent_prop:count_effort:Extraversion,33.100000,0.271075,29.120604,525.0,1994.250000,0.083594


## Train model with top features

In [7]:
top_columns = ['selfish', 'time_spent_prop']
x, y = df_to_xy(df, select_columns=top_columns, min_target=20, max_target=180)


In [8]:
split = int(x.shape[0] * 0.6)
xTrain, yTrain = x[:split], y[:split]
xTest, yTest = x[split:], y[split:]


In [9]:
model = AcceptanceModel()
model.fit(xTrain, yTrain)

In [10]:
from models.metrics import gain_mean, avg_loss_ratio
yPred = model.predict(xTest)
print("Mean gain: ", gain_mean(yTest, yPred))
print("AVG loss ratio: ", avg_loss_ratio(yTest, yPred))

Mean gain:  97.65625
AVG loss ratio:  0.16550664472329668


In [11]:
print("Unique predicted values: ", np.unique(yPred))

Unique predicted values:  [ 95. 100. 105. 110.]


## Generate data for the survey

In [14]:
# Read and sanitize the data
df_test = pd.read_excel("../data/HH_SURVEY1/UG_HH_NEW_continuous_no200_test.xls")

#TODO: add 'prop' to drop_cols?
drop_cols = ['prop', 'other_prop', 'other_resp']
df_test = df_test[[col for col in df_test if col not in drop_cols]]

df_features, df_y = df_to_xydf(df_test, select_columns=top_columns)
predictions = model.predict(df_features.values).astype(int)

df_final = df_test[top_columns].copy()
#RESCALE FEATURES
df_final['pred_min_offer'] = predictions.ravel()
df_final['min_offer'] = df_y['min_offer']
df_final.head()

,selfish,time_spent_prop,pred_min_offer,min_offer
0,25,58000,105,100
1,30,66000,105,120
2,30,38000,100,100
3,15,154000,135,90
4,15,83000,114,100


In [15]:
df_final.to_excel("../data/HH_SURVEY1/UG_HH_NEW_continuous_no200_test_PRED.xls", index=False)